In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
print(tf.__version__)
import warnings
warnings.filterwarnings('ignore')

2.11.0


In [2]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [3]:
ds = pd.read_csv('train_data_fixed.csv')
numConversion = ["left", "right", "center", "auth", "liberal"]
#lambda function to convert x to lowercase and strip all spaces
ds['mainlean'] = ds.mainlean.apply(lambda x: numConversion.index(x.lower().strip()))
ds.dtypes
df=ds.sample(frac=1)

In [4]:
df.head(1)

,tweet,mainlean,secondlean
25724,I had a great time welcoming NISD faculty and ...,0,NaN


In [5]:
split = round(len(df)*0.8)
train_tweets = df['tweet'][:split]
train_label = df['mainlean'][:split]
test_tweets = df['tweet'][split:]
test_label = df['mainlean'][split:]

In [6]:
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []
for row in train_tweets:
    training_sentences.append(str(row))
for row in train_label:
    training_labels.append((row))
for row in test_tweets:
    testing_sentences.append(str(row))
for row in test_label:
    testing_labels.append((row))

In [7]:
print((training_labels[0]))

0


In [8]:
vocab_size = 60000
embedding_dim = 18
max_length = 280
trunc_type = 'post'
oov_tok = '<OOV>'
padding_type = 'post'

In [9]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

In [10]:
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
testing_sentences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sentences, maxlen=max_length)

In [11]:
import math
print(math.sqrt(max_length*5))

37.416573867739416


In [15]:
tf.keras.backend.clear_session()
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=int(max_length)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 280, 18)           1080000   
                                                                 
 global_average_pooling1d (G  (None, 18)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout (Dropout)           (None, 18)                0         
                                                                 
 dense (Dense)               (None, 25)                475       
                                                                 
 dropout_1 (Dropout)         (None, 25)                0         
                                                                 
 dense_1 (Dense)             (None, 5)                 130       
                                                        

In [16]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)
poliIdent = model.fit(padded, training_labels_final, epochs=8, validation_data=(testing_padded, testing_labels_final), callbacks=[es_callback])

Epoch 1/8
3525/3525 [==============================] - 31s 9ms/step - loss: 0.7133 - accuracy: 0.5924 - val_loss: 0.5180 - val_accuracy: 0.7624
Epoch 2/8
3525/3525 [==============================] - 31s 9ms/step - loss: 0.4606 - accuracy: 0.7984 - val_loss: 0.4189 - val_accuracy: 0.8141
Epoch 3/8
3525/3525 [==============================] - 31s 9ms/step - loss: 0.3815 - accuracy: 0.8431 - val_loss: 0.3792 - val_accuracy: 0.8381
Epoch 4/8
3525/3525 [==============================] - 31s 9ms/step - loss: 0.3369 - accuracy: 0.8657 - val_loss: 0.3657 - val_accuracy: 0.8477
Epoch 5/8
3525/3525 [==============================] - 31s 9ms/step - loss: 0.3029 - accuracy: 0.8818 - val_loss: 0.3782 - val_accuracy: 0.8400


In [14]:
model.save("poliIdent")

INFO:tensorflow:Assets written to: poliIdent\assets


INFO:tensorflow:Assets written to: poliIdent\assets
